Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('./whiskey-reviews-ds11/train.csv')
test = pd.read_csv('./whiskey-reviews-ds11/test.csv')

In [2]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [3]:
features = 'description'
target = 'ratingCategory'

X_train = train[features]
y_train = train[target]

X_test = test[features]

train.shape, X_train.shape

((4087, 3), (4087,))

### Define Pipeline Components

In [4]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
vect = TfidfVectorizer(
    stop_words='english',
    ngram_range=(1,2)
)
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [18]:
parameters = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df': (0.2, 3),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-2, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:   31.2s
[Parallel(n_jobs=-2)]: Done  80 out of  80 | elapsed:   53.2s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [26]:
grid_search.best_params_

{'clf__max_depth': 20, 'vect__max_df': 0.75, 'vect__min_df': 3}

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [19]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [20]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [21]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [22]:
subNumber = 0

In [23]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [9]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)

vect = TfidfVectorizer(
    stop_words='english',
    ngram_range=(1,2)
)

clf = RandomForestClassifier()

params = { 
    'lsi__svd__n_components': [10,100],
    'lsi__vect__max_df':[.9, 1.0],
#    'clf__n_estimators':[5,10,20]
}

In [10]:
lsi = Pipeline([
    ('vect', vect),
    ('svd', svd)
])

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [11]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe,params, cv=5, n_jobs=4, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:  1.7min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

In [12]:
grid_search.best_params_

{'lsi__svd__n_components': 100, 'lsi__vect__max_df': 1.0}

### Make a Submission File

In [13]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [14]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [15]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [19]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
# subNumber = 1
submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [1]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [20]:
# Continue Word Embedding Work Here
import spacy
nlp = spacy.load("en_core_web_md")

In [21]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [22]:
X = get_word_vectors(X_train)

In [23]:
X[0]

array([-6.80197403e-02,  2.10571989e-01, -1.26131907e-01, -1.00589722e-01,
        1.11971639e-01,  6.87979087e-02, -7.58958831e-02, -7.88916126e-02,
       -4.13076393e-02,  1.78384101e+00, -1.15093589e-01,  9.56213623e-02,
       -2.31543370e-02, -1.32735327e-01, -8.63617957e-02, -1.51824549e-01,
       -2.95691881e-02,  1.04987657e+00, -6.52607232e-02, -3.57119106e-02,
       -1.97695196e-02, -9.43115354e-03, -2.97431294e-02, -3.89569104e-02,
       -7.73228705e-02, -4.24622037e-02, -4.46286015e-02,  2.09163874e-02,
        1.03280038e-01, -1.53810874e-01, -1.44276004e-02,  4.62238006e-02,
        2.18380280e-02, -3.37490290e-02,  3.84492725e-02,  5.33151673e-04,
        2.70788111e-02, -1.94602422e-02, -5.14513366e-02,  5.02043106e-02,
       -3.67092006e-02,  3.73126455e-02,  1.05339840e-01, -9.75339487e-02,
        6.59741163e-02,  1.31081909e-01, -8.36109966e-02, -6.15613982e-02,
        6.17238879e-02,  2.46760584e-02, -5.13718240e-02,  4.19777371e-02,
       -7.22259469e-03, -

In [24]:
rfc = RandomForestClassifier()

In [25]:
rfc.fit(X, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

### Make a Submission File

In [27]:
# Predictions on test sample
Xtest = get_word_vectors(test['description'])
pred = rfc.predict(Xtest)

In [28]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [29]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [30]:
subNumber

2

In [31]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

In [39]:
import basilica
from dotenv import load_dotenv
import os

load_dotenv()

BASILICA_KEY=os.getenv('BASILICA_KEY')

In [41]:
with basilica.Connection(BASILICA_KEY) as connection:
    embeddings = connection.embed_sentences(X_train, 'product-reviews')

type(embeddings)

generator

In [44]:
embeds_list = list(embeddings)
len(embeds_list)

4087

In [45]:
len(X_train)

4087

In [48]:
train_embeds = embeds_list

with basilica.Connection(BASILICA_KEY) as connection:
    embeddings = connection.embed_sentences(X_test, 'product-reviews')

test_embeds = list(embeddings)
len(test_embeds)

1022

In [49]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV

gbc = GradientBoostingClassifier(random_state=42)

params = {
    'n_estimators': [100, 200],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 5],
    'max_depth': [2, 3, 5],
}

rsc = RandomizedSearchCV(
    estimator=gbc,
    param_distributions=params,
    verbose=1,
    n_jobs=4,
    random_state=42
)

In [50]:
rsc.fit(train_embeds, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

In [57]:
print(subNumber)

def make_submission(pred, n):
    submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
    submission['ratingCategory'] = submission['ratingCategory'].astype('int64')
    csv_name = f'submission{n}.csv'
    submission.to_csv(csv_name, index=False)
    print(csv_name)


3


In [55]:
rfc2 = RandomForestClassifier()
rfc2.fit(train_embeds, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [58]:
pred = rfc2.predict(test_embeds)
make_submission(pred, subNumber)
subNumber += 1

submission3.csv


In [59]:
from sklearn.linear_model import LogisticRegression

In [60]:
lr = LogisticRegression()
lr.fit(train_embeds, y_train)
pred = lr.predict(test_embeds)
make_submission(pred, subNumber)
subNumber += 1

submission4.csv


/Users/guest/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [61]:
lr = LogisticRegression(max_iter=1000)
lr.fit(train_embeds, y_train)
pred = lr.predict(test_embeds)
make_submission(pred, subNumber)
subNumber += 1

submission5.csv


In [ ]:
gbc = GradientBoostingClassifier(random_state=42, verbose=10)
gbc.fit(train_embeds, y_train)
pred = gbc.predict(test_embeds)
make_submission(pred, subNumber)
subNumber += 1

In [67]:
subNumber += 1

In [68]:
rfc = RandomForestClassifier(random_state=42, n_jobs=4)

params = {
    'n_estimators': [50, 100, 500],
    'max_depth': [None, 50, 100],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 7],
    'max_features': [None, 'auto', 'log2']
}

rscv = RandomizedSearchCV(
    estimator=rfc,
    param_distributions=params,
    verbose=10,
    n_jobs=4,
    random_state=42
)

rscv.fit(train_embeds, y_train)
pred = rscv.predict(test_embeds)
make_submission(pred, subNumber)
subNumber += 1

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed: 32.6min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed: 39.8min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed: 64.0min
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed: 66.6min
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed: 69.8min
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 76.0min
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed: 102.6min finished


submission7.csv


### [ryan herr's submission stacking](https://github.com/jcs-lambda/DS-Unit-2-Kaggle-Challenge/blob/master/module4-classification-metrics/LS_DS_224_assignment.ipynb)

In [69]:
files = [f'submission{i}.csv' for i in range(subNumber)]

target = 'ratingCategory'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

make_submission(majority_vote, 99)

submission99.csv


In [78]:
ensemble['majority'] = majority_vote
ensemble['majority'] = ensemble['majority'].astype('int64')
ensemble.head(20)

,ratingCategory,ratingCategory,ratingCategory,ratingCategory,ratingCategory,ratingCategory,ratingCategory,ratingCategory,majority
0,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,0,0,0,1
2,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1
5,1,1,1,1,1,1,1,1,1
6,1,1,1,1,1,1,1,1,1
7,1,1,1,1,1,1,2,1,1
8,1,1,1,1,0,1,1,1,1
9,1,1,1,1,1,1,1,1,1


In [76]:
ensemble.dtypes

ratingCategory      int64
ratingCategory      int64
ratingCategory      int64
ratingCategory      int64
ratingCategory      int64
ratingCategory      int64
ratingCategory      int64
ratingCategory      int64
majority          float64
dtype: object

In [79]:
y_train.value_counts()

1    2881
0    1141
2      65
Name: ratingCategory, dtype: int64

In [80]:
ensemble['majority'].value_counts()

1    885
0    137
Name: majority, dtype: int64

In [82]:
ensemble.columns = [f'sub_{i}' for i in range(subNumber)] + ['majority']

for column in ensemble.columns:
    print(column)
    print(ensemble[column].value_counts())
    print()

sub_0
1    1001
0      21
Name: sub_0, dtype: int64

sub_1
1    979
0     43
Name: sub_1, dtype: int64

sub_2
1    923
0     99
Name: sub_2, dtype: int64

sub_3
1    910
0    112
Name: sub_3, dtype: int64

sub_4
1    774
0    243
2      5
Name: sub_4, dtype: int64

sub_5
1    763
0    254
2      5
Name: sub_5, dtype: int64

sub_6
1    840
0    171
2     11
Name: sub_6, dtype: int64

sub_7
1    882
0    140
Name: sub_7, dtype: int64

majority
1    885
0    137
Name: majority, dtype: int64



# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 75% accuracy on the Kaggle competition. Once you have achieved 75% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?

### Sentiment Analysis

Sentiment analysis is using machine learning techniques like NLP (natural language processing) on a corpus of opinion based documents, such as product reviews or social media posts.

Document classification refers to a wide array of text analysis, of which sentiment analysis is one. Document classification includes many things outside the scope of sentiment analysis, like classifying music or images.

Sentiment analysis is used to classify documents as they regard to human expressions of emotions. Basic sentiment analysis can classify documents as 'positive', 'negative', or 'neutral'. More advanced sentiment analysis tries to expand those categories: 'happy', 'angry', 'sad', etc.

Creating well labeled sentiment data is an intensive manual process that should be done by an expert in the problem domain. There are ways to automate it, but that can only make the job a little easier.